In [ ]:
from irrep.bandstructure import BandStructure
from fractions import Fraction
import sympy
from wannierberri.symmetry.sawf import SymmetrizerSAWF as SAWF


from wannierberri.symmetry.projections_searcher import EBRsearcher
from wannierberri.symmetry.projections import Projection, ProjectionsSet


print("calculating symmetrizer")

path = "../1.diamond/data/"

bandstructure = BandStructure(prefix=path + "di-irred", code="espresso",
                            Ecut=100, include_TR=False, irreducible=True)
spacegroup = bandstructure.spacegroup
# spacegroup.show()


In [ ]:

try:
    symmetrizer = SAWF().from_npz("diamond.sawf.npz")
except FileNotFoundError:
    symmetrizer = SAWF().from_irrep(bandstructure, irreducible=True)
    symmetrizer.to_npz("diamond.sawf.npz")


In [ ]:


trial_projections = ProjectionsSet()

x, y, z = sympy.symbols('x y z')
F12 = Fraction(1, 2)
F14 = Fraction(1, 4)
F18 = Fraction(1, 8)
WP = [[0, 0, 0], [x, 0, 0], [F12, F12, F12], [F14, F14, F14], [F18, F18, F18], [0, x, z]]
# in principle, those should be all wyckoff position for the spacegroup
# but we will only consider a few random positions
positions = [",".join(str(y) for y in x) for x in WP]
print(positions)


for p in positions:
    for o in ['s','p','d']:
        proj = Projection(position_sym=p, orbital=o, spacegroup=spacegroup)
        trial_projections.add(proj)

print("trial_projections: ")
print(trial_projections.write_with_multiplicities(orbit=False))
print("#################\n"*3)

ebrsearcher = EBRsearcher(
    symmetrizer=symmetrizer,
    trial_projections_set=trial_projections,
    froz_min=-20,
    froz_max=25,
    outer_min=-20,
    outer_max=25,
    debug=False
)


combinations = ebrsearcher.find_combinations(num_wann_max=10)

print (f"the following prijections are compatible with the irreps of the DFT bands in the specified window")
for c in combinations:
    print(("+" * 80 + "\n") * 2)
    print(trial_projections.write_with_multiplicities(c))
    newset = trial_projections.get_combination(c)
    newset.join_same_wyckoff()
    newset.maximize_distance()
    print(newset.write_wannier90(mod1=True))
